In [25]:
import os
import glob
import pandas as pd
import re

In [26]:
conjunto_dados = pd.DataFrame()

SPEAKER_REGEX = re.compile(r'[A|O] Sr\.ª?( [\w\-,]+)+([\-\w]+)?( \([\w\s\-]+\))*(?=:)')
MULTILINE_SPEAKER_REGEX = re.compile("^" + SPEAKER_REGEX.pattern, flags=re.M) 

for file in os.scandir("./discursos"):
    speech_by_speaker = []

    with open(file) as fp:
        content = fp.read()

        #1) remover datas espalhadas, páginas e afins espalhadas pelos documentos (por causa da coversao PDF para TXT)

        datas_texto = re.compile("\d{1,2} \w+ \w+ \w+ \d{4}")
        datas_texto.sub(" ", content)

        series_numeros = re.compile("[A-Z]+ SÉRIE \W NÚMERO \d+")
        series_numeros.sub(" ", content)

        #2) criar dicionario de intervencoes para criar dataframe

        #2.1) identificar os intervenientes e intervencoes correspondentes

        deputadxs = MULTILINE_SPEAKER_REGEX.finditer(content) 
        deputadxs = [match.group() for match in deputadxs]

        deputadxs = [deputadxs[0]]

        # print(deputadxs)

        content_on_one_line = ''.join(content.splitlines())

        for speaker in deputadxs:
            content_split_by_speaker = content_on_one_line.split(speaker + ':')
            # Get rid of everything before they speak
            content_split_by_speaker.pop(0)

            # what_they_say = [speech for speech in speaker_regex]
            # for each line in content_split_by_speaker
            # use the speaker regex to find the next speaker and get everything before that
            for line in content_split_by_speaker:
                start_of_next_speaker = SPEAKER_REGEX.finditer(line)

                # either an index or None
                # if it's None, either:
                # - the regex isn't accurate enough; or
                # - the speaker is the only person to speak]
                next_speaker_indexes = [match.start() for match in start_of_next_speaker]
                next_speaker_index = next_speaker_indexes[0] if len(next_speaker_indexes) > 0 else None

                speech = line[0:next_speaker_index] if next_speaker_index is not None else line
                speech_by_speaker.append((speaker, speech))

                # if "):" in speech:
                #     print(speech)
        
        df = pd.DataFrame.from_records(speech_by_speaker, columns=["speaker", "speech"])#.reset_index()
        df['filename'] = os.path.basename(file)
        #print(df)
                
        conjunto_dados = pd.concat([conjunto_dados, df])
            
conjunto_dados
    

,speaker,speech,filename
0,O Sr. Presidente,"— Sr.as e Srs. Deputados, Sr.as e Srs. Funcio...",darl13sl03n050.txt
1,O Sr. Presidente,"— Srs. Deputados, vamos passar ao primeiro po...",darl13sl03n050.txt
2,O Sr. Presidente,"— Sr.ª Deputada, inscreveram-se quatro Srs. D...",darl13sl03n050.txt
3,O Sr. Presidente,"— Para pedir esclarecimentos, tem a palavra o...",darl13sl03n050.txt
4,O Sr. Presidente,"— Peço-lhe para concluir, Sr. Deputado, pois ...",darl13sl03n050.txt
...,...,...,...
52,O Sr. Presidente,"— Fica registado, Sr. Deputado. Vamos votar o...",darl13sl03n058.txt
53,O Sr. Presidente,"— Tem a palavra, Sr. Deputado João Oliveira.",darl13sl03n058.txt
54,O Sr. Presidente,"— Fica, então, registado que o PCP votou cont...",darl13sl03n058.txt
55,O Sr. Presidente,"— Faça favor, Sr. Deputado.",darl13sl03n058.txt
